In [1]:
from pygments.styles import get_style_by_name

get_style_by_name("colorful")

# styles

pygments.styles.colorful.ColorfulStyle

In [2]:
import itertools

import numpy as np
import pandas as pd

In [3]:
responses_raw_df = pd.read_csv("DataCamp/Quiz-1-2020-11-19/Quiz DataCamp 1.csv")
weights_raw_df = pd.read_csv("weights.csv")
ref_raw_df = pd.read_csv("DataCamp/Quiz-1-2020-11-19/Quiz DataCamp 1 - Ref.csv")

In [4]:
weights_df = weights_raw_df.set_index("Coef").rename(
    columns={"Right": True, "Wrong": False}
)
ref_df = ref_raw_df.set_index("Q_id")

In [5]:
ref_df

,Response
Q_id,
Q1,Classification
Q2,Dimensionality reduction
Q3,The data are not sampled in a representative way
Q4,All of the above
Q5,To set the hyperparameters


In [6]:
weights_df

,False,True
Coef,,
5,-20,20
4,-6,19
3,0,18
2,2,17
1,3,16
0,4,13


In [7]:
responses_raw_df.columns

Index(['Timestamp', 'First name', 'Last name',
       'What is a common supervised task?', 'Confidence',
       'What is Principal Component Analysis (PCA) mainly used for in ML?',
       'Confidence.1', 'What does sampling bias mean?', 'Confidence.2',
       'What is regularization doing to the model?', 'Confidence.3',
       'What's the purpose of a validation set?', 'Confidence.4',
       'Student ID'],
      dtype='object')

In [8]:
def make_headers(basename, old_titles):
    return [f"{basename}{el}" for el in range(1, len(old_titles) // 2 + 1)]


def mapping(from_, to_):
    return {old: new for old, new in zip(from_, to_)}

In [9]:
titles = responses_raw_df.columns[3:-1]

# question_ids = [f"Q{el}" for el in range(1, len(titles)//2 + 1)]

question_ids = make_headers("Q", titles)
coefficient_ids = [f"C{el[1:]}" for el in question_ids]
response_ids = [f"R{el[1:]}" for el in question_ids]
values_ids = [f"V{el[1:]}" for el in question_ids]

short_titles = list(itertools.chain.from_iterable(zip(question_ids, coefficient_ids)))

# short2long = {short: long for short, long in zip(short_titles, titles)}
# long2short = {long: short for short, long in short2long.items()}

In [10]:
# mapping(titles, to=short_titles)

In [11]:
txt = "abc"
# txt.replace("a", "d")

In [12]:
responses_df = responses_raw_df.rename(columns=mapping(titles, to_=short_titles))

In [13]:
responses_df[question_ids].head()

,Q1,Q2,Q3,Q4,Q5
0,Classification,Dimensionality reduction,The data are not sampled in a representative way,It makes the model less prone to overfitting,To set the hyperparameters
1,Classification,Dimensionality reduction,All of the above,It makes the model less prone to overfitting,To validate the model before sending it to pro...
2,Classification,Dimensionality reduction,The data are not sampled in a representative way,It makes the model less prone to overfitting,To set the hyperparameters
3,Classification,Dimensionality reduction,The data are not sampled in a representative way,All of the above,To set the hyperparameters
4,Clustering,Dimensionality reduction,All of the above,It makes the model less prone to overfitting,To validate the model before sending it to pro...


In [14]:
(responses_df[coefficient_ids].dtypes == np.dtype("int")).all()

True

In [15]:
def comb(df1, df2, link_df):

    comb_df = df1.copy()

    for val in link_df:
        tmp_df = df2.copy()
        tmp_df = tmp_df.applymap(lambda x: link_df.loc[x, val])
        comb_df = comb_df.replace({val: np.nan}).combine_first(tmp_df)

    return comb_df

In [16]:
val1_df = pd.DataFrame({"A": ["aa", "bb"], "B": ["bb", "aa"]})
print(val1_df)

    A   B
0  aa  bb
1  bb  aa


In [17]:
val2_df = pd.DataFrame({"A": ["xx", "yy"], "B": ["zz", "yy"]})
print(val2_df)

    A   B
0  xx  zz
1  yy  yy


In [18]:
aaxx_df = pd.DataFrame(
    {"aa": [10, 20, 30], "bb": [-10, -20, -30]}, index=["xx", "yy", "zz"]
)
print(aaxx_df)

    aa  bb
xx  10 -10
yy  20 -20
zz  30 -30


In [19]:
# %%timeit  # 4.66 ms

comb(val1_df, val2_df, aaxx_df)

,A,B
0,10.0,-30.0
1,-20.0,20.0


In [20]:
# %%timeit  # 230 µs

vals = aaxx_df.lookup(val2_df.values.flatten(), val1_df.values.flatten())
vals

array([ 10, -30, -20,  20])

In [21]:
res = pd.DataFrame(vals.reshape(val1_df.shape), columns=val1_df.columns)
res

,A,B
0,10,-30
1,-20,20


In [22]:
# %%timeit  # 595 µs

aaxx_df.loc[val2_df.values.flatten(), val1_df.values.flatten()]
# vals = np.diag()
# res = pd.DataFrame(vals.reshape(val1_df.shape), columns=val1_df.columns)
# res

,aa,bb,bb,aa
xx,10,-10,-10,10
zz,30,-30,-30,30
yy,20,-20,-20,20
yy,20,-20,-20,20


In [23]:
# %%timeit  # 4.45 ms

res = (
    pd.concat([val1_df, val2_df])
    .groupby(level=0)
    .agg(tuple)
    .applymap(lambda x: aaxx_df.loc[x[1], x[0]])
)
res

,A,B
0,10,-30
1,-20,20


In [24]:
pd.concat([val1_df, val2_df])

,A,B
0,aa,bb
1,bb,aa
0,xx,zz
1,yy,yy


In [25]:
pd.concat([val1_df, val2_df]).groupby(level=0).agg(tuple)

val2_df.stack()

0  A    xx
   B    zz
1  A    yy
   B    yy
dtype: object

In [26]:
pd.concat([val1_df, val2_df]).groupby

<bound method DataFrame.groupby of     A   B
0  aa  bb
1  bb  aa
0  xx  zz
1  yy  yy>

In [27]:
aaxx_df.stack()
val2_df.stack()

0  A    xx
   B    zz
1  A    yy
   B    yy
dtype: object

In [28]:
# %%timeit  # 1.9 ms

s = aaxx_df.stack().reindex(
    pd.MultiIndex.from_arrays([val2_df.stack(), val1_df.stack()])
)
s

xx  aa    10
zz  bb   -30
yy  bb   -20
    aa    20
dtype: int64

In [29]:
val1_df.stack()

0  A    aa
   B    bb
1  A    bb
   B    aa
dtype: object

In [30]:
val2_df.stack()

0  A    xx
   B    zz
1  A    yy
   B    yy
dtype: object

In [31]:
# %%timeit  # 1.1 ms

pd.MultiIndex.from_arrays([val2_df.stack(), val1_df.stack()])

MultiIndex([('xx', 'aa'),
            ('zz', 'bb'),
            ('yy', 'bb'),
            ('yy', 'aa')],
           )

In [32]:
val_df = s.values.reshape(val1_df.shape)
val_df

array([[ 10, -30],
       [-20,  20]])

In [33]:
df = pd.DataFrame(
    {"col": ["A", "A", "B", "B"], "A": [80, 23, np.nan, 22], "B": [80, 55, 76, 67]}
)
df

,col,A,B
0,A,80.0,80
1,A,23.0,55
2,B,NaN,76
3,B,22.0,67


In [34]:
df.melt("col")

,col,variable,value
0,A,A,80.0
1,A,A,23.0
2,B,A,NaN
3,B,A,22.0
4,A,B,80.0
5,A,B,55.0
6,B,B,76.0
7,B,B,67.0


In [35]:
melt_df = aaxx_df.reset_index().melt("index")
melt_df

,index,variable,value
0,xx,aa,10
1,yy,aa,20
2,zz,aa,30
3,xx,bb,-10
4,yy,bb,-20
5,zz,bb,-30


In [36]:
# melt_df.loc[((melt_df["index"] == val2_df.values.flatten()) & (melt_df["variable"] == val1_df.values.flatten()))]

In [37]:
def lookup(df, row_labels, col_labels) -> np.ndarray:
    """
    Label-based "fancy indexing" function for DataFrame.
    Given equal-length arrays of row and column labels, return an
    array of the values corresponding to each (row, col) pair.

    Parameters
    ----------
    row_labels : sequence
        The row labels to use for lookup.
    col_labels : sequence
        The column labels to use for lookup.

    Returns
    -------
    numpy.ndarray
        The found values.
    """
    row_idx = df.index.get_indexer(row_labels)
    col_idx = df.columns.get_indexer(col_labels)
    flat_index = row_idx * len(df.columns) + col_idx
    return df.values.flat[flat_index]

In [38]:
# %%timeit  # 156 µs
lookup(aaxx_df, val2_df.values.flatten(), val1_df.values.flatten())

array([ 10, -30, -20,  20])

In [39]:
# %%timeit 148 µs

# lookup(aaxx_df, val2_df.values.flatten(), val1_df.values.flatten())
row_idx = aaxx_df.index.get_indexer(val2_df.values.flatten())
row_idx
col_idx = aaxx_df.columns.get_indexer(val1_df.values.flatten())
col_idx
flat_index = row_idx * len(aaxx_df.columns) + col_idx
flat_index
aaxx_df.values.flat[flat_index]
aaxx_df.values.flat[[0, 1, 2, 3, 4, 5]]

array([ 10, -10,  20, -20,  30, -30])

In [40]:
def grade_1(responses_df, matches_df, weights_df):

    grades_df = matches_df.rename(columns=mapping(response_ids, to_=values_ids))
    for is_right in weights_df:
        values_df = responses_df[coefficient_ids].rename(
            columns=mapping(coefficient_ids, to_=values_ids)
        )
        values_df = values_df.applymap(lambda x: weights_df.loc[x, is_right])
        grades_df = grades_df.replace({is_right: np.nan}).combine_first(values_df)

    return grades_df

In [41]:
def grade_2(responses_df, matches_df, weights_df):
    values = weights_df.lookup(
        responses_df[coefficient_ids].values.flatten(),
        matches_df.values.flatten(),
    )
    return pd.DataFrame(
        values.reshape(matches_df.shape), columns=matches_df.columns
    ).rename(columns=mapping(response_ids, to_=values_ids))

In [117]:
def grade_3(responses_df, matches_df, weights_df):

    return (
        pd.concat(
            [
                matches_df.rename(columns=mapping(response_ids, to_=values_ids)),
                responses_df[coefficient_ids].rename(
                    columns=mapping(coefficient_ids, to_=values_ids)
                ),
            ]
        )
                .groupby(level=0)
                .agg(tuple)
                .applymap(lambda x: weights_df.loc[x[1], bool(x[0])])
    )

In [131]:
weights_df.stack()

Coef       
5     False   -20
      True     20
4     False    -6
      True     19
3     False     0
      True     18
2     False     2
      True     17
1     False     3
      True     16
0     False     4
      True     13
dtype: int64

In [137]:
weights_df.reset_index().melt("Coef")

,Coef,variable,value
0,5,False,-20
1,4,False,-6
2,3,False,0
3,2,False,2
4,1,False,3
5,0,False,4
6,5,True,20
7,4,True,19
8,3,True,18
9,2,True,17


In [139]:
matches_df.stack()

0   R1     True
    R2     True
    R3     True
    R4    False
    R5     True
          ...  
26  R1    False
    R2     True
    R3    False
    R4    False
    R5    False
Length: 135, dtype: bool

In [97]:
def grade_4(responses_df, matches_df, weights_df):
    s = weights_df.stack().reindex(
        pd.MultiIndex.from_arrays([responses_df[coefficient_ids].stack(), matches_df.stack()])
    )
    # grades_df[:] = s.values.reshape(matches_df.shape)

    # return pd.DataFrame(s.values.reshape(val1_df.shape), columns=val1_df.columns)
    return pd.DataFrame(
        s.values.reshape(matches_df.shape), columns=matches_df.columns
    ).rename(columns=mapping(response_ids, to_=values_ids))

---

In [44]:
ref_df.transpose()

Q_id,Q1,Q2,Q3,Q4,Q5
Response,Classification,Dimensionality reduction,The data are not sampled in a representative way,All of the above,To set the hyperparameters


In [45]:
matches_df = responses_df[question_ids].eq(ref_df.Response)
matches_df.rename(columns=mapping(question_ids, to_=response_ids), inplace=True)
matches_df.head()

,R1,R2,R3,R4,R5
0,True,True,True,False,True
1,True,True,False,False,False
2,True,True,True,False,True
3,True,True,True,True,True
4,False,True,False,False,False


In [46]:
# responses_df.Q5 == ref_df.loc["Q5", "Response"]

In [47]:
# responses_df[coefficient_ids].head()

In [48]:
# responses_df["C1"].map(weights_df)

In [49]:
df1 = pd.DataFrame({"A": ["a", "b", "a"], "B": ["b", "b", "a"]})
df2 = pd.DataFrame({"A": [True, False, True], "B": [True, False, False]})

In [50]:
# print(df1.to_markdown())

In [51]:
conv_df = pd.DataFrame({True: [1, 2], False: [3, 4]}, index=["a", "b"])

In [52]:
df = pd.DataFrame({"A": [1, 2, 1], "B": [2, 2, 1]})

In [53]:
conv_s = pd.Series([1, 2], index=["a", "b"])

In [54]:
# df1["A"].map(conv_s)

In [127]:
# %%timeit  # grade_1: 10.3 ms, grade_2: 815 µs, grade_3: 10.9 ms, grade_4: 2.5 ms

# grades_df = matches_df.rename(columns=mapping(response_ids, to=values_ids))

# for is_right in [True, False]:
#     values_df = responses_df[coefficient_ids].rename(
#         columns=mapping(coefficient_ids, to=values_ids)
#     )
#     values_df = values_df.applymap(lambda x: weights_df.loc[x, is_right])
#     grades_df = grades_df.replace({is_right: np.nan}).combine_first(values_df)

grades_df = grade_4(responses_df, matches_df, weights_df)
grades_df

,V1,V2,V3,V4,V5
0,20,19,19,-6,20
1,20,19,-6,-6,-6
2,20,20,20,-20,20
3,20,20,18,18,20
4,-6,20,0,-6,0
5,20,20,20,20,20
6,3,19,0,18,0
7,20,20,18,0,0
8,20,20,3,-20,19
9,20,20,18,-6,19


In [119]:
# grades_df = matches_df.copy()

# for is_right in [True, False]:
#     values_df = responses_df[coefficient_ids].applymap(lambda x: weights_df.loc[x, is_right])
#     values_df.rename(columns={f"C{i}": f"Q{i}" for i in range(1, 5+1)}, inplace=True)
#     grades_df = grades_df.replace({is_right: np.nan}).combine_first(values_df)

In [120]:
# grades_df.mean(axis="columns").clip(0, 20)

In [121]:
# grades_df.mean(axis="columns").clip(0, 20).clip(baseline)

In [122]:
baseline = (matches_df * 18).mean(axis="columns")

In [123]:
results_df = responses_df.join(matches_df * 1).join(grades_df)
results_df["Total"] = grades_df.mean(axis="columns").clip(0, 20)
results_df["Baseline"] = (matches_df * 18).mean(axis="columns")
results_df["Total_adjusted"] = results_df["Total"].clip(results_df["Baseline"])

In [124]:
res_val_ids = itertools.chain.from_iterable(zip(response_ids, values_ids))

output_ids = ["Last name"] + list(res_val_ids) + ["Total", "Baseline", "Total_adjusted"]

# output_ids = ["Last name", "R1", "R2", "R3", "R4", "R5", ]
results_df[output_ids]

,Last name,R1,V1,R2,V2,R3,V3,R4,V4,R5,V5,Total,Baseline,Total_adjusted
0,Camps,1,20,1,19,1,19,0,-6,1,20,14.4,14.4,14.4
1,Kazi Aoual,1,20,1,19,0,-6,0,-6,0,-6,4.2,7.2,7.2
2,Jaminet,1,20,1,20,1,20,0,-20,1,20,12.0,14.4,14.4
3,Arora,1,20,1,20,1,18,1,18,1,20,19.2,18.0,19.2
4,Abderrahmane,0,-6,1,20,0,0,0,-6,0,0,1.6,3.6,3.6
5,LAMALLE,1,20,1,20,1,20,1,20,1,20,20.0,18.0,20.0
6,wen,0,3,1,19,0,0,1,18,0,0,8.0,7.2,8.0
7,Cintra,1,20,1,20,1,18,0,0,0,0,11.6,10.8,11.6
8,Jemli,1,20,1,20,0,3,0,-20,1,19,8.4,10.8,10.8
9,Soulard,1,20,1,20,1,18,0,-6,1,19,14.2,14.4,14.4


In [125]:
results_df[output_ids].describe()

,R1,V1,R2,V2,R3,V3,R4,V4,R5,V5,Total,Baseline,Total_adjusted
count,27.000000,27.000000,27.0,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.814815,16.074074,1.0,19.666667,0.518519,8.148148,0.481481,4.407407,0.481481,7.814815,11.370370,11.866667,12.481481
std,0.395847,8.507580,0.0,0.877058,0.509175,12.461336,0.509175,15.280101,0.509175,12.446924,6.117492,4.663442,4.951227
min,0.000000,-6.000000,1.0,16.000000,0.000000,-20.000000,0.000000,-20.000000,0.000000,-20.000000,0.000000,3.600000,3.600000
25%,1.000000,20.000000,1.0,20.000000,0.000000,0.000000,0.000000,-6.000000,0.000000,0.000000,7.200000,7.200000,7.600000
50%,1.000000,20.000000,1.0,20.000000,1.000000,16.000000,0.000000,0.000000,0.000000,3.000000,13.000000,14.400000,14.400000
75%,1.000000,20.000000,1.0,20.000000,1.000000,19.000000,1.000000,19.000000,1.000000,19.500000,15.600000,14.400000,15.600000
max,1.000000,20.000000,1.0,20.000000,1.000000,20.000000,1.000000,20.000000,1.000000,20.000000,20.000000,18.000000,20.000000


|       |        R1 |       V1 |   R2 |        V2 |        R3 |        V3 |        R4 |        V4 |        R5 |        V5 |    Total |   Baseline |   Total_adjusted |
|:------|----------:|---------:|-----:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|---------:|-----------:|-----------------:|
| count | 27        | 27       |   27 | 27        | 27        |  27       | 27        |  27       | 27        |  27       | 27       |   27       |         27       |
| mean  |  0.814815 | 16.0741  |    1 | 19.6667   |  0.518519 |   8.14815 |  0.481481 |   4.40741 |  0.481481 |   7.81481 | 11.3704  |   11.8667  |         12.4815  |
| std   |  0.395847 |  8.50758 |    0 |  0.877058 |  0.509175 |  12.4613  |  0.509175 |  15.2801  |  0.509175 |  12.4469  |  6.11749 |    4.66344 |          4.95123 |
| min   |  0        | -6       |    1 | 16        |  0        | -20       |  0        | -20       |  0        | -20       |  0       |    3.6     |          3.6     |
| 25%   |  1        | 20       |    1 | 20        |  0        |   0       |  0        |  -6       |  0        |   0       |  7.2     |    7.2     |          7.6     |
| 50%   |  1        | 20       |    1 | 20        |  1        |  16       |  0        |   0       |  0        |   3       | 13       |   14.4     |         14.4     |
| 75%   |  1        | 20       |    1 | 20        |  1        |  19       |  1        |  19       |  1        |  19.5     | 15.6     |   14.4     |         15.6     |
| max   |  1        | 20       |    1 | 20        |  1        |  20       |  1        |  20       |  1        |  20       | 20       |   18       |         20       |